# Standard Features
## Working with single measurement scans

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
import KiMoPack.plot_func as pf
from importlib import reload
%matplotlib tk

This notebook is an introduction to transient absorption spectroscopy. <br>
In contrast to the other tutorials we are using artificial data for this analysis. <br> 
For this training we will be using 'con_1.SIA' which is clean data with some added noise.<br>
The following datasets 'con_1.SIA','con_2.SIA','con_3.SIA','con_4.SIA' are based on the same data<br> 
for which additional complications such as noise, chirp and crossphase modulation was added.<br>
These datasets were generated by creating a reaction of the type A->B->C and spectra as shown in this image:
![Chirp](img/Intro_tutorial.png "Data_content")

In [ ]:
filenames =['con_1.SIA','con_2.SIA','con_3.SIA','con_4.SIA']               # set name of the file to fit
#If the path is only one folder one can just give the function the name, otherwise one has to give the absolute path
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')  # set path to file to fit
ta=pf.TA(filenames[0],path=filepath)
plt.close('all')
ta.Plot_RAW(0)

In [ ]:
# Can't remember the commands?
ta()

We investigate the 2D matrix using the option to click on the data. We plot this spectrum twice, once in logaritmic scale and once in linear scale. Making the plot clickable means that the values where you click are shown. 
I usually use this method to define where I do would like to show the plots.

In [ ]:
#the 2d Matrix is plot 0 (check ta.Plot_RAW? for a tutorial)
ta.log_scale=False # This is the default
ta.Plot_RAW(0,print_click_position=True)
ta.log_scale=True   # This scales the 2d plot into log scale.
ta.Plot_RAW(0,print_click_position=True)

Setting these values I plot the same matrix again and select the interesting wavelength using the same process

In [ ]:
ta.bordercut=[400,975]
ta.timelimits=[-0.2,500]
plt.close('all')
ta.Plot_RAW(0,print_click_position=True)

Based on the shown wavelength I set the interesting wavelength and plot it a last time and select the intersting time points

In [ ]:
ta.rel_wave=[430,487,525,640,720,820,900,950]
plt.close('all')
ta.Plot_RAW(0,print_click_position=True)

again, the clicked points are converted to time points and we have set the intersting time points.

In [ ]:
ta.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]

Now we are ready to make an overview plot of the data

In [ ]:
ta.log_scale=False
ta.Save_Powerpoint(save_Fit=False,title='Tutorial plot')

For fitting usually the kinetic plots are a good choice to select the times I believe are a good starting point for a fit. In this process i look for all the clear decays and click on them. I also look for if there is signal before time=0 (nothing here) and if there is a signal after all is decayed (nothing here)

In [ ]:
ta.Plot_RAW(1,print_click_position=True)

Based on these clicks we create a model. In general starting with exponential decays is a good idea. Here we choose three distinct times in lines 4-6, add instrument parameter (fixed for now) in rows 8 and 9. 
Important is the line 10-12. This is a simple trick to "freeze" all parameter. This allows us to check how good are our starting parameter.
Line 14 is triggering the fit. And line 17 plotting the results

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
###-------Adding instrument parameter, here frozen---------------
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
if 1:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

As everything looks pretty good we simply copy paste the same code, but now set the if switch in line 9 to "0" to disable this loop and premit the optimization 

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
###-------Adding instrument parameter, here frozen---------------
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

The result is refining and getting much better, but the change to the starting values is very small. So we repeat the same fit but allow the time resolution to be adjusted optimizing the instrument parameter. (line 6 and 7)

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=True)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=True)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

From this we fint and fix the instrument resolution and the arrival time of the laser.

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

A quick look on the DAS it is clear that they are the change of the spectra, to get species associated spectra we change to target analysis and the related model A->B->C by changing the model to 'consecutive' but keeping the same parameter.

In [ ]:
plt.close('all')
ta.Plot_fit_output(0) 

In [ ]:
ta.mod='consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=-0.01837,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

the longer timepoints are very well represented, but there are still some errors in the early times. So we permit the laser arrival time, and the instrument response function free again and change the model to "full_consecutive". (In case you are in a hurry, set the vary of 't0' and 'resolution' to False. 

In [ ]:
ta.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                     
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

This results in a 60 percent improvement of the R2 factor. But still all the DAS contain a contribution of the ground state bleach. So we are adding the ground state explicitely to try to separate the contributions. We lock the resolution and the laser arrival time "I0" to speed up the fit

In [ ]:
ta.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.100143,vary=True)                  
par.add('k1',value=1/2.496702,vary=True)             
par.add('k2',value=1/39.963222,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
par.add('explicit_GS')

ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

Now we can compare the time evolution of the components with the species associated spectra that were put into the data.
![Chirp](img/Intro_tutorial.png "Data_content")

In [ ]:
plt.close('all')
ta.Plot_fit_output(0)

In [ ]:
# save the results
plt.close('all')
ta.Save_Powerpoint(title='Tutorial plot after Fit')

It is a very imporant step to check the confidence interval, that unfortunately does take quite some time. (typically 100x the time for  a single optimization. It is generally a good idea to save the project before you do that with ta.Save_project() to not loose the prior work. The following cell shows the result of this run. The file "con_1_solved.hdf5" contains the project with the result for you to inspect.

In [ ]:
reload(pf)
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')
ta=pf.TA('con_1_solved.hdf5',path=filepath)
#ta.Print_Results()
ta.filename='con_1.hdf5'
ta.Save_Powerpoint(save_RAW=False,savetype='pdf')

In [ ]:
fig,ax=plt.subplots()

In [ ]:
ax.imshow?

In [ ]:
import matplotlib.font_manager
from IPython.core.display import HTML
def make_html(fontname):
    return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)
code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])
HTML("<div style='column-count: 2;'>{}</div>".format(code))